## Method -2 : Using Docker container

I've installed docker-ce on my local

In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import os
import json

In [2]:
!pip install -q requests

import requests

### Save model in the required format to get protobuf file

In [3]:
h5model = keras.models.load_model("C:/Users/AChowdhry/Desktop/cookpad\model/0000001/tomato_model.h5",compile=False)


In [7]:
tf.saved_model.save(h5model, "C:/Users/AChowdhry/Desktop/cookpad/model/0000002/")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: C:/Users/AChowdhry/Desktop/cookpad/model/0000002/assets


### Download the TensorFlow Serving Docker image and repo

In [4]:
!git clone --recursive https://github.com/tensorflow/serving

fatal: destination path 'serving' already exists and is not an empty directory.


In [6]:
!docker pull tensorflow/serving

Using default tag: latest
latest: Pulling from tensorflow/serving
Digest: sha256:a94b7e3b0e825350675e83b0c2f2fc28f34be358c34e4126a1d828de899ec44f
Status: Image is up to date for tensorflow/serving:latest
docker.io/tensorflow/serving:latest


### Mounting the on the container so TF Serving can read the model from inside the container

Using os.system to run the command as background processes are not supported. 

Found that running this using os.system causes entering a NET_LOG loop and remains that way not allowing execution of subsequent commands. Hence I terminated the os.system cell, started docker container locally using docker desktop.


This command will start a docker container, deploy the model and make a REST endpoint available to get predictions.

In [30]:
commandstring = "docker run -p 8501:8501 --name cookpad --mount type=bind,source=C:/Users/AChowdhry/Desktop/cookpad/model/,target=/models/saved_model.pb -e MODEL_NAME=saved_model.pb -t tensorflow/serving &"

In [31]:
os.system(commandstring)

0

### Sending three samples for inference - taken from inference.py file

In [21]:
predict_dataset = [
    [5.1, 3.3, 1.7, 0.5, ],
    [5.9, 3.0, 4.2, 1.5, ],
    [6.9, 3.1, 5.4, 2.1]
]

In [22]:
#json format for api calls
data = json.dumps({"signature_name": "serving_default", "instances": predict_dataset})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ... , 0.5], [5.9, 3.0, 4.2, 1.5], [6.9, 3.1, 5.4, 2.1]]}


In [23]:
class_names = ['Plum', 'Cherry', 'BeefSteak']

In [24]:
headers = {"content-type": "application/json"}

### Post query which will send the served model the above instances or examples and get back predictions.

In [25]:
json_response = requests.post('http://localhost:8501/v1/models/saved_model.pb:predict', data=data, headers=headers,verify=False)

In [26]:
apipredictions = json.loads(json_response.text)['predictions']


In [27]:
for i in range(len(apipredictions)):
    print(class_names[np.argmax(apipredictions[i])], np.argmax(apipredictions[i]))

Plum 0
Cherry 1
BeefSteak 2
